# Flickr data Mining script

Zhongrui Ning

## 1. import your flickr api

In [1]:
# Import Flickr API
import os
import json
import requests
import pandas as pd
from datetime import datetime, timedelta
from time import sleep
from concurrent.futures import ThreadPoolExecutor, as_completed
api_key ='f33d5e60551c0b35cfc78eb4aeee186c'
api_secret = '1e96a5669c9157aa'

In [ ]:
api_key = 'c2d834ecbe14197d2d21bcb0e473a2e1'
api_secret = 'cb9eebb9aac6ac51'
# back up key

In [2]:
api_key = 'e7e7d9aa60b883213ae27da208f7ee4a'
api_secret = '9807436dcdccbe3a'

## 2. Get the user data in the U.S. from 2015 to 2023

In [ ]:
import os
import json
import requests
import shutil
import pandas as pd
from datetime import datetime, timedelta
from time import sleep
from concurrent.futures import ThreadPoolExecutor, as_completed

# Define the parameters
EXTRAS = "url_o,license,geo,lat,lon,tags,description,date_taken"
PER_PAGE = 250
BASE_URL = "https://www.flickr.com/services/rest/"
TEMP_FILE = 'flickr_fetch_temp.json'
FINAL_FILE = 'flickr_images_sorted.csv'

def daterange(start_date, end_date, delta_days=10):
    while start_date < end_date:
        yield start_date
        start_date += timedelta(days=delta_days)

def fetch_photos(min_date, max_date, retries=5):
    params = {
        "method": "flickr.photos.search",
        "api_key": api_key,
        "min_taken_date": min_date,
        "max_taken_date": max_date,
        "extras": EXTRAS,
        "format": "json",
        "nojsoncallback": "1",
        "per_page": PER_PAGE,
        "page": 1,  # Start at page 1
        "bbox": "-125.001650,24.9493,-66.9326,49.5904"  # Coordinates for the US
    }
    all_photos = []
    while True:
        for attempt in range(retries):
            try:
                response = requests.get(BASE_URL, params=params)
                
                if response.status_code == 200:
                    data = response.json()
                    photos = data['photos']['photo']
                    all_photos.extend(photos)
                    
                    # Get the total number of pages and stop if we've reached the last page
                    total_pages = data['photos']['pages']
                    current_page = params['page']
                    
                    if current_page >= total_pages:
                        return all_photos
                    
                    params['page'] += 1  # Move to the next page
                    break
                else:
                    print(f"Error: Received status code {response.status_code}. Retrying...")
                    sleep(2 ** attempt)
            except requests.exceptions.RequestException as e:
                print(f"Network error: {e}. Retrying...")
                sleep(2 ** attempt)
        else:
            break
    return all_photos

def load_checkpoint():
    if os.path.exists(TEMP_FILE):
        try:
            with open(TEMP_FILE, 'r') as f:
                data = json.load(f)
                return data['date_ranges'], data['photos']
        except json.JSONDecodeError as e:
            print(f"Warning: Temp file {TEMP_FILE} is corrupted. Starting fresh. Error: {e}")
            # 如果文件损坏，删除它并返回空结果
            os.remove(TEMP_FILE)
            return [], []
    return [], []

def save_checkpoint(date_ranges, photos, all_photos):
    # Combine existing photos with new photos and remove duplicates based on 'id'
    combined_photos = all_photos + photos
    unique_photos = {photo['id']: photo for photo in combined_photos}.values()

    temp_file = TEMP_FILE + '.tmp'
    # Save unique photos to a temporary file
    with open(temp_file, 'w') as f:
        json.dump({'date_ranges': date_ranges, 'photos': list(unique_photos)}, f)
    
    # After successful write, replace the original file
    shutil.move(temp_file, TEMP_FILE)

def save_to_csv(photos):
    if os.path.exists(FINAL_FILE):
        existing_df = pd.read_csv(FINAL_FILE)
    else:
        existing_df = pd.DataFrame()

    new_df = pd.DataFrame(photos)
    combined_df = pd.concat([existing_df, new_df]).drop_duplicates(subset='id')

    combined_df.to_csv(FINAL_FILE, index=False)

def process_date_range(min_date_str, max_date_str):
    photos = fetch_photos(min_date_str, max_date_str)
    return photos

def main():
    start_date = datetime.strptime("2015-01-01", "%Y-%m-%d")
    end_date = datetime.strptime("2023-12-31", "%Y-%m-%d")

# load checkpoint
    completed_date_ranges, all_photos = load_checkpoint()

    date_ranges = []
    for single_date in daterange(start_date, end_date, delta_days=10):
        min_date_str = single_date.strftime("%Y-%m-%d")
        max_date_str = (single_date + timedelta(days=10)).strftime("%Y-%m-%d")
        if [min_date_str, max_date_str] not in completed_date_ranges:
            date_ranges.append((min_date_str, max_date_str))

    if not date_ranges:
        print("All date ranges have been processed.")
        return

    # using multiple threads to process the date ranges
    with ThreadPoolExecutor(max_workers=5) as executor:
        futures = {executor.submit(process_date_range, dr[0], dr[1]): dr for dr in date_ranges}
        for future in as_completed(futures):
            try:
                photos = future.result()
                if photos:
                    all_photos.extend(photos)
                    completed_date_ranges.append(futures[future])
                    print(f"Total photos fetched so far: {len(all_photos)}")

                    save_checkpoint(completed_date_ranges, photos, all_photos)
                    save_to_csv(photos)
            except Exception as e:
                print(f"Error occurred: {e}")
            sleep(1)

if __name__ == "__main__":
    main()

All date ranges have been processed.


## Collecting the climate data and calculating anomalies and extreme weather